In [1]:
import numpy as np

In [2]:
input = '1'+'2'

In [3]:
T = len(input)

In [4]:
vocabs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

stoi = {s: i for i, s in enumerate(vocabs)}
itos = {i: s for i, s in enumerate(vocabs)}

stoi

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [5]:
[stoi[s] for s in input]

[1, 2]

In [6]:
import numpy as np

W_emb = np.array([
    [0, ],  # 0
    [1, ],  # 1
    [2, ],  # 2
    [3, ],  # 3
    [4, ],  # 4
    [5, ],  # 5
    [6, ],  # 6
    [7, ],  # 7
    [8, ],  # 8
    [9, ],  # 9
], dtype=float)


assert W_emb.shape[0] == len(vocabs)
emb_dim = W_emb.shape[1]
W_emb.round()

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]])

In [7]:
x = np.array([W_emb[stoi[token]] for token in input])
x

array([[1.],
       [2.]])

In [8]:
head_count = 1
d_head = emb_dim
d_model = d_head * head_count
d_model

1

In [9]:
W_Q = np.zeros((emb_dim, d_model))
W_K = np.zeros((emb_dim, d_model))
W_V = np.eye(d_model)*2

Q = x @ W_Q
K = x @ W_K
V = x @ W_V

assert Q.shape == (T, d_model)
assert K.shape == (T, d_model)
assert V.shape == (T, d_model)

In [10]:
import torch

M = torch.triu(torch.ones(T, T) * float('-inf'), diagonal=1)
M = M.masked_fill(torch.isnan(M), 0).numpy()
M


array([[  0., -inf],
       [  0.,   0.]], dtype=float32)

In [11]:
QK = (Q @ K.T) / np.sqrt(d_model)
weights = torch.softmax(torch.from_numpy(QK+M), dim=-1)
weights

tensor([[1.0000, 0.0000],
        [0.5000, 0.5000]], dtype=torch.float64)

In [12]:
W_V

array([[2.]])

In [13]:
x

array([[1.],
       [2.]])

In [14]:
V

array([[2.],
       [4.]])

In [15]:
Attention = weights @ V
Attention # Caveat: LayerNorm is not applied.

tensor([[2.],
        [3.]], dtype=torch.float64)

In [16]:
# ff_dim = d_model
ff_dim = d_model * 4

W_1 = np.ones((d_model, ff_dim), dtype=float)
b_1 = np.zeros((ff_dim,), dtype=float)
print('W_1\n', W_1)
print('b_1\n', b_1)
assert W_1.shape == (d_model, ff_dim)
assert b_1.shape == (ff_dim,)

W_2 = np.ones((ff_dim, d_model), dtype=float) / ff_dim
b_2 = np.zeros((d_model,), dtype=float)
print('W_2\n', W_2)
print('b_2\n', b_2)
assert W_2.shape == (ff_dim, d_model)
assert b_2.shape == (d_model,)

FFN_X = torch.relu(torch.from_numpy(Attention.numpy() @ W_1 + b_1)) @ W_2 + b_2
assert FFN_X.shape == Attention.shape == (T, d_model)
print('FFN(X)\n', FFN_X.round())

W_1
 [[1. 1. 1. 1.]]
b_1
 [0. 0. 0. 0.]
W_2
 [[0.25]
 [0.25]
 [0.25]
 [0.25]]
b_2
 [0.]
FFN(X)
 tensor([[2.],
        [3.]], dtype=torch.float64)


In [17]:
W_Z = np.ones((d_model, len(vocabs)), dtype=float)  # Last linear layer that converts to logits

In [18]:
next_token_logits = (FFN_X @ W_Z)[-1]
next_token_logits

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=torch.float64)

In [19]:
next_token = itos[torch.argmax(torch.softmax(next_token_logits, dim=-1)).item()]
next_token

'0'

In [20]:
# Result
print(f'input: {input} => next token: {next_token}')

input: 12 => next token: 0
